In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader

In [3]:
class Inception_Mod(nn.Module):
    def __init__(self, prev_layer):
        super(Inception_Mod, self).__init__()  
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=prev_layer, out_channels=6, kernel_size=1)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=prev_layer, out_channels=6, kernel_size=1),
            nn.Conv2d(in_channels=6, out_channels= 6, kernel_size=3, stride=1, padding=1)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels=prev_layer, out_channels=6, kernel_size=1),
            nn.Conv2d(in_channels=6, out_channels=6, kernel_size=5, padding = 2, stride=1)
        )
        self.block4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1),
            nn.Conv2d(in_channels=prev_layer, out_channels=6,kernel_size=1, padding=1)
        )
        
    def forward(self, x):
        first_block = self.block1(x)
        second_block = self.block2(x)
        third_block = self.block3(x)
        fourth_block = self.block4(x)

        outputs_concat = torch.cat([first_block, second_block, third_block, fourth_block], dim  = 0)
        return outputs_concat

In [5]:
def create_inceptionMod(prev_layer):
        inception_mod = Inception_Mod(prev_layer.shape[1])
        output = inception_mod(prev_layer)
        return output

def createBranches(data):
        x_view = torch.unsqueeze(data[0], dim = 0)
        y_view = torch.unsqueeze(data[1], dim = 0)
        return x_view, y_view
def mergeXY_views(x_view, y_view):
        return torch.cat([x_view, y_view], dim=0)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# image size = 1 x256 x 256
random_tensor = torch.randn(1, 1, 256, 256)

class CVN_Net(nn.Module):
    def __init__(self):
        super(CVN_Net,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6,  kernel_size=7, stride=2)
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride = 2)        
        self.lrn_norm = nn.LocalResponseNorm(size=5,  alpha=0.0001, beta=0.75)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels= 12, kernel_size=1)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=6, kernel_size=3)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pool(x)
        x = self.lrn_norm(x)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.lrn_norm(x)
        x = self.max_pool(x)

        inception_mod_output1 = create_inceptionMod(x)
        inception_mod_output2 = create_inceptionMod(inception_mod_output1)
        x = self.max_pool(inception_mod_output2)
        inception_mod_output3 = create_inceptionMod(x)
        return inception_mod_output3


model = CVN_Net().to(device=device)
output = model(random_tensor)

print(output.shape)

torch.Size([64, 6, 14, 14])


[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


## Testing entire Architecture

In [7]:
hep_data = torch.randn(2,1, 256, 256)
x_data, y_data = createBranches(hep_data)


x_model = CVN_Net().to(device=device)
y_model = CVN_Net().to(device=device)
x_output = x_model(x_data)
y_output = y_model(y_data)
print(f'Shape of X_View Data: {x_output.shape}')
print(f'Shape of Y_View Data: {y_output.shape}')
concatXY_views = torch.concat([x_output, y_output], dim = 0)
print(concatXY_views.shape)
final_inception_output = create_inceptionMod(concatXY_views)
avg_pooling = nn.AvgPool2d(kernel_size=(6,5))
output = avg_pooling(final_inception_output)
softmax = nn.LogSoftmax(dim = 0)
output = softmax(output)

Shape of X_View Data: torch.Size([64, 6, 14, 14])
Shape of Y_View Data: torch.Size([64, 6, 14, 14])
torch.Size([128, 6, 14, 14])
